# Overview

## Getting started

Primarily designed for the Scipp library, Plopp offers a number of different ways to plot data from a Scipp `Variable`, `DataArray`, or a `Dataset`.
It can however also plot raw [NumPy arrays, as well as Xarray and Pandas](numpy-pandas-xarray.ipynb) data structures.
It uses the `matplotlib` graphing library to do so, as well as the `pythreejs` project for 3D visualizations.

<div class="alert alert-warning">

**Making plots interactive**

Matplotlib makes static plots by default in Jupyter notebooks. To enable interactive plots, use
```
%matplotlib widget
```
at the start of your notebook
(see [here](https://matplotlib.org/stable/users/explain/backends.html#the-builtin-backends)
for more details on Matplotlib backends).
    
</div>

Plotting functionality is available in two different ways:

- using the [plot()](../../generated/plopp.plot.rst) free function
- using the `.plot()` method on a Scipp object (variable, data array or dataset)

The difference between the two possible plot functions is that the free function can accept more input types than just the Scipp objects.
It can also plot NumPy, Pandas and Xarray data structures, as well as Python dicts of Scipp variables or data arrays.
For Scipp objects, the produced plot will be the same with either approach: 
Internally, the `.plot()` method just forwards the Scipp object to the free function `plot()`.

Consider two data arrays storing 1-D data:

In [ ]:
import numpy as np
import scipp as sc
import plopp as pp

size = 50
rng = np.random.default_rng(seed=0)
x = sc.linspace('x', 0.0, 2.0, num=size, unit='m')
y = sc.linspace('y', 0.0, 1.0, num=5, unit='us')
temp1 = sc.array(dims=['x'], values=rng.random(size), unit='K')
temp1 += sc.linspace('x', 100, 105, num=size, unit='K')
da1 = sc.DataArray(temp1, coords={'x': x})
da1.name = 'temp1'  # Data array name is optional and will be used as a label
temp2 = sc.array(dims=['x'], values=rng.random(size), unit='K')
temp2.variances = 0.5 * (temp2.values + 0.5)
temp2 += sc.linspace('x', 99, 102, num=size, unit='K')
da2 = sc.DataArray(temp2, coords={'x': x})

The information in a data array or dataset is typically enough to create meaningful plots:

In [ ]:
da1.plot()

In the plot above, the `x` dimension has an associated `x` coordinate and its values are used to label the ticks on the horizontal axis.
The coordinate name and the unit (here: `m`) are used as a label for the horizontal axis.
In a 1-D plot the unit of the data values (here `K`) labels the vertical axis.

Multiple data arrays can be plotted by passing a Python dict to the `plot` function.
This example also illustrates how a data array with uncertainties results in a plot with error bars:

In [ ]:
pp.plot({'temp1': da1, 'temp2': da2})

When the data arrays are part of a data group or a dataset, we can plot them directly:

In [ ]:
dg = sc.DataGroup({'temp1': da1, 'temp2': da2})
dg.plot()

## Plotting slices or items of a dataset/data group

The usual indexing and slicing can be used to create plots of slices of data, or plots of individual items from a dataset or a data group.

### Plot a single entry

In [ ]:
dg['temp2'].plot()

### Plot a slice range

In [ ]:
dg['x', 10:30].plot()

## 2-D data

### Creating a 2-D plot

2-D data arrays can be plotted directly, just like 1-D data:

In [ ]:
size = 50
rng = np.random.default_rng(seed=0)
x = sc.linspace('x', 1.0, 300.0, num=size, unit='m')
time = sc.linspace('time', 1.0, 200.0, num=2 * size, unit='us')
temp = sc.array(dims=['x', 'time'], values=rng.random((size, 2 * size)), unit='K')
temp += sc.linspace('x', 100, 105, num=size, unit='K')
da = sc.DataArray(temp, coords={'x': x, 'time': time})
da.name = 'temperature'  # Data array name is optional and will be used as a label

This result in the following 2-D plot:

In [ ]:
da.plot()

### Plot a 1-D slice of 2-D data

When slicing without a range, the dimensionality reduces.
This can be used to, e.g., plot a 1-D slice through 2-D data:

In [ ]:
da['time', 4].plot()

### Transpose axes

To control which dimensions are shown along which axes of the matplotlib figure transpose data before calling `plot`:

In [ ]:
da.transpose().plot()

When the Matplotlib "widget" backend is activate it is also possible to transpose the axes via a button in the toolbar.

## Logarithmic scale

Data can be plotted on a logarithmic scale on one or both axes.
For the independent axis (the coordinate axis, i.e., the horizontal axis) this can be set using the `scale` option:

In [ ]:
da1.plot(scale={'x': 'log'})

In [ ]:
da.plot(scale={'x': 'log', 'time': 'log'})

Note the the keys in the `scale` dict are dimension labels and *not* "x" and "y" as Matplotlib would refer to its axes.

For the dependent axis (the data axis, i.e., vertical axis) use the `norm` option:

In [ ]:
(100 * (da1 - da1.min())).plot(norm='log')